In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb


2024-09-16 09:57:36.700571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 09:57:36.700666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 09:57:36.860350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")
print("wandb")
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune_Llama2_7B', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
wandb


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elbrahimimeriem (elbrahimimeriem-university hassan 2). Use `wandb login --relogin` to force relogin


In [5]:
base_model = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "/kaggle/input/dataset-agriculture-pc-k18/dataset_agriculture_pc_18k.xlsx"
new_model = "llama-2-7b-ft-agri"

In [6]:
torch_dtype = torch.float16
attn_implementation = "eager"
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [8]:
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [1]:

import pandas as pd
from datasets import Dataset

dataset = pd.read_excel(dataset_name)
dataset


,Prompt,Context,Prompt_context
0,Prunier: Taille de formation,Il faut intervenir sur les pruniers les 7 prem...,Prunier: Taille de formation \n Il faut interv...
1,Quels outils utiliser pour la taille d'un prun...,"Pour effectuer la taille d'un prunier, il faut...",Quels outils utiliser pour la taille d'un prun...
2,Comment procéder à la taille d'un prunier ?,"Pour tailler un prunier efficacement, il suffi...",Comment procéder à la taille d'un prunier ? \n...
3,Semis : comment bien choisir son noyau de pêche ?,"Avant de planter un noyau, choisir une pêche d...",Semis : comment bien choisir son noyau de pêch...
4,Pourquoi installent-on des pêchers dans les vi...,Les pêches mûrissent au moment des vendanges. ...,Pourquoi installent-on des pêchers dans les vi...
...,...,...,...
18204,Pourquoi choisir une zone large et ensoleillée...,Il est important de choisir un endroit large a...,Pourquoi choisir une zone large et ensoleillée...
18205,Quel est le sol approprié pour la culture des ...,Si votre sol a été utilisé pour cultiver des f...,Quel est le sol approprié pour la culture des ...
18206,Comment protéger les plants de citrouilles des...,Il est important de choisir un endroit pour pl...,Comment protéger les plants de citrouilles des...
18207,Quel est le meilleur moment pour planter des c...,Les citrouilles sont très sensibles aux gelées...,Quel est le meilleur moment pour planter des c...


In [12]:
from datasets import Dataset
import pandas as pd

def transform_dataset(dataset):
    transformed_data = []
    system_message ="""
    Tu es un assistant virtuel dédié à l'agriculture au Maroc. Tu aides les agriculteurs en leur fournissant  des réponses détaillées et adaptées aux conditions locales.
    """ 
    for index, example in dataset.iterrows():
        prompt = str(example['Prompt']).strip() if pd.notna(example['Prompt']) else ""
        response = str(example['Context']).strip() if pd.notna(example['Context']) else ""

        # Apply the new template (assuming [PROMPT] and [RESPONSE] placeholders)
        transformed_example = f'{system_message}<s>[PROMPT] {prompt} [/PROMPT] {response} </s>'

        transformed_data.append(transformed_example)

    return transformed_data

# Appliquer la fonction au DataFrame nettoyé
transformed_data = transform_dataset(dataset)

# Créer un DataFrame pandas à partir des données transformées
df_transformed = pd.DataFrame({'text': transformed_data})

# Utiliser Dataset.from_pandas pour convertir en Dataset Hugging Face
dataset = Dataset.from_pandas(df_transformed)

# Vérifier le résultat
print(dataset['text'][233])
from sklearn.model_selection import train_test_split
# Diviser les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(df_transformed, test_size=0.009, random_state=42)

# Convertir les DataFrames pandas en Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)



    Tu es un assistant virtuel dédié à l'agriculture au Maroc. Tu aides les agriculteurs en leur fournissant  des réponses détaillées et adaptées aux conditions locales.
    <s>[PROMPT] La stratification des noyaux d'olive : pas-à-pas [/PROMPT] Choisir des olives très mûres, noires, molles et prêtes à tomber. Gratter le noyau avec du papier de verre ou une lime à ongles. Tremper les graines dans une eau tiède pendant 48h. Mélanger les noyaux dans un pot rempli de sable légèrement humide. Enterrer le pot au pied d'un mur exposé au nord recouvert d'un grillage et des feuilles mortes. </s>


In [13]:

training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    save_steps=0
) 


# Define SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/18045 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
1805,1.344500,1.170764
3610,1.213500,1.096950
5415,1.307800,1.042395
7220,1.026300,1.001693


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=9022, training_loss=1.0628780049191984, metrics={'train_runtime': 17452.8801, 'train_samples_per_second': 1.034, 'train_steps_per_second': 0.517, 'total_flos': 2.0129608926353818e+17, 'train_loss': 1.0628780049191984, 'epoch': 0.999944582986977})

In [15]:
model.save_pretrained("llama2_agri")
tokenizer.save_pretrained("llama2_agri")
#trainer.model.config.push_to_hub("laila1234/fine-tuned-llama3-300l-without config")
tokenizer.push_to_hub("MeelUnv/llama2_agri")
model.push_to_hub("MeelUnv/llama2_agri")
config = trainer.model.config
config.push_to_hub("MeelUnv/llama2_agri")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/684M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MeelUnv/llama2_agri/commit/c9b2bf6ba182d98e3163f29bbc5e175b5fdf639b', commit_message='Upload config', commit_description='', oid='c9b2bf6ba182d98e3163f29bbc5e175b5fdf639b', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
from datetime import datetime

# Obtenir l'heure actuelle
heure_actuelle = datetime.now().time()

# Afficher l'heure actuelle
print("L'heure actuelle est :", heure_actuelle)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                max_length=550,
    
    temperature=0.7,
    top_p=0.9, 
    no_repeat_ngram_size=3,  # Avoid repeating n-grams
    num_beams=5,  # Use beam search for completeness
    early_stopping=True # Ajouter une pénalité de répétition  
    
)

# Disable logging to prevent excessive output
logging.set_verbosity(logging.CRITICAL)
q="quelles régions sont appropriées pour la culture des fèves au maroc"
text=pipe(f"<s>[INST] {q} [/INST]")
a=text[0]['generated_text']
a=a.split("[/INST]")[1].split("[/INST:")[0]
print(a)
heure_actuelle = datetime.now().time()
print("L'heure actuelle est :", heure_actuelle)


L'heure actuelle est : 15:31:08.598977
 Les principales régions de culture de la fève au Maroc sont le Gharb, le Souss-Massa, le Haouz, le Saïs et le Doukkala.[1]
L'heure actuelle est : 15:33:04.136822
